In [3]:
from os import listdir
from os.path import isfile, join

statdir = "playerstats"
files = [join(statdir, f) for f in listdir(statdir) if isfile(join(statdir, f))]

In [5]:
import json

jsons = []
for jf in files:
    with open(jf) as f:
        jsons.append()

In [12]:
stats = []
for playerstats in jsons:
    for playerstat in playerstats:
        stats.append(playerstat)

In [107]:
import numpy as np

ignorethese = ('MatchID', 'Lane', 'Role')

def filterdata(data, lane, role):
    return filter(lambda stat: (lane == "ANY" or stat["Lane"] == lane) and 
                                    (role == "ANY" or stat["Role"] == role), 
                                   data)

def createAvgStats(stats, lane, role):
    avgstats = {}
    filteredstats = filterdata(stats, lane, role)
    for stat in filteredstats:
        for key in stat:
            if key not in ignorethese:
                if key in avgstats:
                    avgstats[key] += stat[key]
                else:
                    avgstats[key] = stat[key]
    for key in avgstats:
        avgstats[key] /= float(len(filteredstats))
    avgstats["Lane"] = lane
    avgstats["Role"] = role
    return avgstats

def createStatsDistribution(stats, lane, role):
    statdistribution = {}
    filteredstats = filterdata(stats, lane, role)
    for stat in filteredstats:
        for key in stat:
            if key not in ignorethese:
                if key in statdistribution:
                    statdistribution[key].append(stat[key])
                else:
                    statdistribution[key] = [stat[key]]
        
    statdistribution["Lane"] = lane
    statdistribution["Role"] = role
    return statdistribution

def createStdStats(stats, lane, role):
    sd = createStatsDistribution(stats, lane, role)
    
    stds = {}
    for key in sd:
        if key == "Lane" or key == "Role":
            stds[key] = sd[key]
        else:
            stds[key] = np.std(sd[key])
    
    return stds

In [113]:
avgstatdata = []
avgstatdata.append(createAvgStats(stats, "TOP", "SOLO"))
avgstatdata.append(createAvgStats(stats, "JUNGLE", "NONE"))
avgstatdata.append(createAvgStats(stats, "MIDDLE", "SOLO"))
avgstatdata.append(createAvgStats(stats, "BOTTOM", "DUO_CARRY"))
avgstatdata.append(createAvgStats(stats, "BOTTOM", "DUO_SUPPORT"))
avgstatdata.append(createAvgStats(stats, "ANY", "ANY"))

In [114]:
with open('player_stats_mean.json', 'w') as f:
    json.dump(avgstatdata, f)

In [115]:
stddata = []
stddata.append(createStdStats(stats, "TOP", "SOLO"))
stddata.append(createStdStats(stats, "JUNGLE", "NONE"))
stddata.append(createStdStats(stats, "MIDDLE", "SOLO"))
stddata.append(createStdStats(stats, "BOTTOM", "DUO_CARRY"))
stddata.append(createStdStats(stats, "BOTTOM", "DUO_SUPPORT"))
stddata.append(createStdStats(stats, "ANY", "ANY"))

In [116]:
with open('player_stats_std.json', 'w') as f:
    json.dump(stddata, f)

In [117]:
avgstatdata[0]["Kills"]

5.471951285229293

In [112]:
stddata[0]["Kills"]

3.7899150219324009